## 1.多层感知机从零实现

In [ ]:
#mlp-scratch ------多层感知机从零实现
import torch
from torch import nn
from d2l import torch as d2l

#数据导入
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

#初始化模型参数
num_inputs,num_outputs,num_hiddens=784,10,256 #输入，输出，隐藏层隐藏单元
W1=nn.Parameter(torch.randn(num_inputs,num_hiddens,requires_grad=True)*0.01) #输入层到隐藏层
b1=nn.Parameter(torch.zeros(num_hiddens,require_grad=True))
W2=nn.Parameter(torch.randn(num_inputs,num_hiddens,require_grad=True)*0.01) #隐藏层到输出层
b2=nn.Parameter(torch.zeros(num_outputs,require_grad=True))

params=[W1,b1,W2,b2]

#激活函数：实现ReLU函数
def relu(X):
    a=torch.zeros_like(X)
    return torch.max(X,a)

#定义模型
def net(X):
    X=X.reshape((-1,num_inputs)) #将每个二维图像转换为一个长度为num_inputs的向量
    H=relu(X@W1+b1) #这里@表示矩阵乘法
    return (H@W2+b2)

#损失函数
loss=nn.CrossEntropyLoss() #使用内置API计算交叉熵损失

#训练：与softmax训练过程完全相同
num_epochs,lr=10,0.1
updater=torch.optim.SGD(params,lr=lr)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,updater)

#模型应用于测试数据
d2l.predict_ch3(net,test_iter)

## 2.多层感知机简洁实现--使用API构建网络

In [ ]:
#mlp-concise ------多层感知机简洁实现--使用API构建网络
import torch
feom torch import nn
from d2l import torch as d2l

#模型
net=nn.Sequential(nn.Flatten(),
                 nn.Linear(784,256),
                 nn.ReLU().
                 nn.Linear(256,10))
def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights);

#交叉熵损失函数
loss=nn.CrossEntropyLoss() 

#训练
batch_size,lr,num_epochs=256,0.1,10
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size) #数据导入
trainer=torch.optim.SGD(net.parameters(),lr=lr) #参数优化
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,trainer)

## 3.多项式拟合--模型复杂性

In [ ]:
import math 
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

#生成数据
max_degree=20 #多项式最大阶数
n_train,n_test=100,100 #训练集和测试集大小
true_w=np.zeros(max_degree) #分配空间
true_w[0:4]=np.array([5,1.2,-3.4,5.6])

features=np.random.normal(size=(n_train+ntest,1)) #训练集和测试集均为100x1的正态分布随机数
np.random.shuffle(features)
poly_features=np.power(features,np.arange(max_degree).reshape(1,-1)) #feature中的数，通过广播机制求每个数的0~19次方，ploy_features.shape=200x20。np.arange(max_degree).reshape(1, -1)将轴数从1调整为2
for i in range(max_degree):
    poly_features[:,i]/=math.gamma(i,1) #自变量的n次方除以对应阶数n的gamma(n)=(n-1)!
labels=np.dot(poly_features,true_w) #各阶数的自变量乘以系数得到输出y。labels维度：(n_train+n_test,)
labels+=np.random.normal(scale=0.1,size=labels.shape) #加上噪声
true_w,features,poly_features,labels=[torch.tensor(x,dtype=torch.float32) 
                                     for x in [true_w,features,poly_features,labels]] #numpy ndarray转换为tensor

#损失评估----评估给定数据集上模型的
def evaluate_loss(net,data_iter,loss): #@save  #输入网络类型，数据集，损失函数
    metric=d2l.Accumulator(2) #构建含2个0元素的列表，第1个元素表示损失总和，第2个元素表示元素数目。 实用程序类Accumulator，用于对多个变量进行累加
    for X,y in data_iter:
        out=net(X) #训练得到预测输出
        y=y.reshape(out.shape)
        l=loss(out,y) #损失函数计算损失
        metrix.add(l.sum(),l.numel()) #numel()返回数组中元素的个数
    return metric[0]/metric[1] #返回训练样本的损失均值

#训练函数
def train(train_features,test_features,train_labels,test_labels,
         num_epochs=400):
    
    #损失函数
    loss=nn.MSELoss() #使用均方差损失函数MSELoss
    
    #网络模型
    input_shape=train_features.shape(-1) #shape[-1]获取shape中的最后一个元素，即样本数量
    net=nn.Sequential(nn.Linear(input_shape,1,bias=False)) ## 不设置偏置，因为已经在多项式特征中实现了偏置
    
    #构建训练/测试数据集
    batch_size=min(10,train_labels.shape[0]) #获取批量大小
    train_iter=d2l.load_array((train_features,train_labels.shape(-1,1),batch_size)) #加载训练数据集
    test_iter=d2l.load_array((test_features,test_labels.reshape(-1,1),batch_size,is_train=False))#加载测试数据集
    
    #优化函数
    trainer=torch.opti.SGD(net.parameters(),lr=0.01) #使用随机梯度下降算法更新参数
    
    #绘制画布、坐标轴、图例
    animator=d2l.Animator(xlabel='epoch',ylabel='loss',yscale='log',xlim=[1,num_epochs],ylim=[le-3,le2],legend=['train','test'])#调用d2l.Animator(),绘制训练过程中训练集和测试集上的epoch-loss图
    
    #训练
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net,train_iter,loss,trainer) #调用训练函数进行训练，不断更新参数--多项式系数
        if epoch==0 or (epoch+1)%20==0: #由训练进度绘制数据点，分别绘制训练集和测试集平均损失。evaluate_loss()计算数据集上的损失均值
            animator.add(epoch+1,(evaluate_loss(net,train_iter,loss),evaluate_loss(net,test_iter,loss)))
    print('weight:',net[0].weight.data.numpy()) #打印多项式系数
    
#三阶多项式函数拟合
train(poly_features[:n_train,:4],poly_features[n_train:,:4],labels[:n_train],labels[n_train:]) #poly_features[:ntrain,:4] 获取前ntrain个样本的前4个特征维度，最高为3阶，即 1, x, x^2/2!, x^3/3!。已定义n_train=100，前100个样本作为训练集，剩余100个样本最为测试集
    
#线性函数拟合(欠拟合)
train(poly_features[:n_train,:2],poly_features[n_train:,:2],labels[:n_train],labels[n_train:]) #只选择前两个特征，最高阶为1，即1, x

#高阶多项式拟合(过拟合)
train(poly_features[:n_train,:],poly_features[n_train:,:],labels[:n_train],labels[n_train:]) #选择所有特征，即包含最高阶数19


## 4.权重衰减